# 原始数据清理及SQL数据导入准备

In [10]:
import pandas as pd
import json
import re

--> 检验数据格式

In [3]:
import os, pathlib

path = "douban_movies_2021.json"
print("exists:", os.path.exists(path))
print("size:", os.path.getsize(path), "bytes")
print("cwd:", pathlib.Path().resolve())

with open(path, "rb") as f:
    head = f.read(200)
print(head[:200])

exists: True
size: 389951705 bytes
cwd: /Users/zhaolong/Documents/<freshgrad/SQL/project
b'[\n  {\n    "movie_id": 1292052,\n    "title": "\xe8\x82\x96\xe7\x94\xb3\xe5\x85\x8b\xe7\x9a\x84\xe6\x95\x91\xe8\xb5\x8e",\n    "original_title": "The Shawshank Redemption",\n    "aka": [\n      "[\'\xe6\x9c\x88\xe9\xbb\x91\xe9\xab\x98\xe9\xa3\x9e(\xe6\xb8\xaf)\'",\n      "\'\xe5\x88\xba\xe6\xbf\x801995(\xe5\x8f\xb0)\'",\n      "\'\xe5\x9c\xb0\xe7\x8b\xb1'


## 1. 原始数据导入及主表构建

In [4]:
raw_df = pd.read_json('douban_movies_2021.json')

In [5]:
movie_cols = [
    'movie_id', 'title', 'original_title',
    'year', 'mainland_pubdate', 'summary', 'record_time'
]
movie_df = raw_df[movie_cols].copy()

In [47]:
movie_df.to_csv('movie.csv', index=False)

In [6]:
def extract_rating(r):
    if pd.isna(r):
        return pd.Series({
            'score': None,
            'rating_count': None,
            'star_1_count': None,
            'star_2_count': None,
            'star_3_count': None,
            'star_4_count': None,
            'star_5_count': None
        })
    score = r.get('score')
    count = r.get('count')
    details = (r.get('raw') or {}).get('details', {})
    return pd.Series({
        'score': score,
        'rating_count': count,
        'star_1_count': int(details.get('1', 0)),
        'star_2_count': int(details.get('2', 0)),
        'star_3_count': int(details.get('3', 0)),
        'star_4_count': int(details.get('4', 0)),
        'star_5_count': int(details.get('5', 0)),
    })

rating_df = raw_df['rating'].apply(extract_rating)
movie_rating_df = pd.concat([raw_df['movie_id'], rating_df], axis=1)

# 再加上 collect_count / wish_count / comments_count / reviews_count
for col in ['collect_count', 'wish_count',
            'comments_count', 'reviews_count']:
    movie_rating_df[col] = raw_df[col]

In [48]:
movie_rating_df.to_csv('movie_rating.csv', index=False)

## 2. 电影类型维度表与关系表的构建

In [11]:
# 1. 先展开 genres（如果本身就是 list，这一步即可；如果是字符串后面再说）
genre_exploded = (
    raw_df[['movie_id', 'genres']]
      .explode('genres')
      .dropna(subset=['genres'])
)

# 2. 定义一个清洗函数，统一去掉 [], 引号 和多余空格
def clean_genre(g):
    if pd.isna(g):
        return None
    s = str(g).strip()
    # 去掉最常见的包裹字符：[ ] ' "
    s = s.strip("[]'\" ")          # 先从两端剥一层
    s = s.replace("[", "").replace("]", "")  # 再把中间残留的彻底去掉
    s = s.strip(" '\"\t\r\n")      # 再次去掉空白和引号
    # 连续空白压缩为一个空格（防止中英文之间有多空格）
    s = re.sub(r"\s+", " ", s)
    return s or None

genre_exploded["genre_clean"] = genre_exploded["genres"].apply(clean_genre)
# 丢掉被清洗成空或 None 的
genre_exploded = genre_exploded.dropna(subset=["genre_clean"])

# 3. 用清洗后的字段生成维度表
genre_dim = (
    genre_exploded[["genre_clean"]]
      .drop_duplicates()
      .reset_index(drop=True)
)
genre_dim["genre_id"] = genre_dim.index + 1
genre_dim = genre_dim.rename(columns={"genre_clean": "genre_name"})

# 4. 生成关系表 movie_genre
movie_genre_df = genre_exploded.merge(
    genre_dim,
    left_on="genre_clean", right_on="genre_name"
)[["movie_id", "genre_id"]].drop_duplicates()

In [49]:
genre_dim.to_csv('genre.csv', index=False)
movie_genre_df.to_csv('movie_genre.csv', index=False)

## 3. 电影国家/地区、语言维度表与关系表的构建

In [30]:
# 专门给国家字段用的拆分+清洗函数
def split_country_tokens(val):
    """
    输入：countries 列中的一个元素（可能是 str / list 的单元）
    输出：一个“干净的国家名”列表
    """
    if pd.isna(val):
        return []
    s = str(val).strip()
    if not s:
        return []

    # 去掉最外层 [], 引号 等
    s = s.strip("[]'\" ")
    s = s.replace("[", "").replace("]", "")
    s = s.strip()

    # 如果整体看起来像日期或纯数字，直接丢弃
    # 例：1958-06-29, 2019, 1997-1-1 等
    if re.fullmatch(r"\d{4}(-\d{1,2}(-\d{1,2})?)?", s):
        return []

    # 先按常见分隔符切一层：逗号 / 中文逗号 / 顿号 / 斜杠
    parts = re.split(r"[、,，/]+", s)
    tokens = []

    for part in parts:
        part = part.strip()
        if not part:
            continue

        # 如果这一段里既有中文又有英文，或者是多段中文中间有空格，
        # 说明其实是多个国家拼在一起，再按空格拆
        if (re.search(r"[\u4e00-\u9fff]", part) and " " in part) or \
           (re.fullmatch(r"[\u4e00-\u9fff\s]+", part) and " " in part):
            sub_parts = [p.strip() for p in part.split(" ") if p.strip()]
            tokens.extend(sub_parts)
        else:
            # 对全英文的部分，不再按空格拆（避免把 "United States" 拆成两块）
            tokens.append(part)

    # 最后再统一清理一遍（去掉多余空白）
    cleaned = []
    for t in tokens:
        t = t.strip(" '\"\t\r\n")
        t = re.sub(r"\s+", " ", t)  # 连续空白压缩成一个空格
        if t:
            cleaned.append(t)

    return cleaned

# =============== 构造 country_exploded ===============

rows = []
for _, row in raw_df[['movie_id', 'countries']].iterrows():
    mid = row['movie_id']
    # countries 字段一般是 list，如果是单个字符串也兼容
    vals = row['countries']
    if isinstance(vals, (list, tuple)):
        src_list = vals
    else:
        src_list = [vals]

    for v in src_list:
        for token in split_country_tokens(v):
            rows.append({'movie_id': mid, 'country_clean': token})

country_exploded = pd.DataFrame(rows)

# 去掉同一部电影里重复的国家
country_exploded = country_exploded.drop_duplicates(
    subset=['movie_id', 'country_clean']
)

# =============== 维度表 country_dim ===============

country_dim = (
    country_exploded[['country_clean']]
    .drop_duplicates()
    .reset_index(drop=True)
)
country_dim['country_id'] = country_dim.index + 1
country_dim = country_dim.rename(columns={'country_clean': 'country_name'})

# =============== 关系表 movie_country_df ===============

movie_country_df = country_exploded.merge(
    country_dim,
    left_on='country_clean', right_on='country_name'
)[['movie_id', 'country_id']].drop_duplicates()


In [50]:
country_dim.to_csv('country.csv', index=False)
movie_country_df.to_csv('movie_country.csv', index=False)

In [34]:
import pandas as pd
import re

# 专门给 languages 用的拆分+清洗函数
def split_language_tokens(val):
    """
    输入：languages 列中的一个元素（可能是 str / list 的单元）
    输出：一个“干净的语言名”列表
    """
    if pd.isna(val):
        return []
    s = str(val).strip()
    if not s:
        return []

    # 去掉最外层 [], 引号 等
    s = s.strip("[]'\" ")
    s = s.replace("[", "").replace("]", "")
    s = s.strip()

    # 如果整体看起来像日期或纯数字，直接丢弃
    # 例：1958-06-29, 2019, 1997-1-1 等
    if re.fullmatch(r"\d{4}(-\d{1,2}(-\d{1,2})?)?", s):
        return []

    # 按常见分隔符切一层：逗号 / 中文逗号 / 顿号 / 斜杠 / 分号 / 竖线
    parts = re.split(r"[、,，/;|]+", s)
    tokens = []

    for part in parts:
        part = part.strip()
        if not part:
            continue

        # 如果这一段只包含中文和空格，且内部有空格，
        # 说明很可能是多个中文语言名拼在一起，例如 "法语 西班牙语"
        if re.fullmatch(r"[\u4e00-\u9fff\s]+", part) and " " in part:
            sub_parts = [p.strip() for p in part.split(" ") if p.strip()]
            tokens.extend(sub_parts)
        else:
            # 对英文多词语言名（含字母）或已经干净的中文名，不再按空格拆
            tokens.append(part)

    # 最后统一清理一次：去掉多余空白、引号等
    cleaned = []
    for t in tokens:
        t = t.strip(" '\"\t\r\n")
        t = re.sub(r"\s+", " ", t)  # 连续空白压成一个空格
        if t:
            cleaned.append(t)

    return cleaned

# =============== 构造 language_exploded ===============

lang_rows = []
for _, row in raw_df[['movie_id', 'languages']].iterrows():
    mid = row['movie_id']
    vals = row['languages']

    # languages 字段有时是 list，有时是单个字符串，这里统一成 list
    if isinstance(vals, (list, tuple)):
        src_list = vals
    else:
        src_list = [vals]

    for v in src_list:
        for token in split_language_tokens(v):
            lang_rows.append({'movie_id': mid, 'language_clean': token})

language_exploded = pd.DataFrame(lang_rows)

# 去掉同一电影里重复的语言
language_exploded = language_exploded.drop_duplicates(
    subset=['movie_id', 'language_clean']
)

# =============== 维度表 language_dim ===============

language_dim = (
    language_exploded[['language_clean']]
    .drop_duplicates()
    .reset_index(drop=True)
)
language_dim['language_id'] = language_dim.index + 1
language_dim = language_dim.rename(columns={'language_clean': 'language_name'})

# =============== 关系表 movie_language_df ===============

movie_language_df = language_exploded.merge(
    language_dim,
    left_on='language_clean', right_on='language_name'
)[['movie_id', 'language_id']].drop_duplicates()


In [51]:
language_dim.to_csv('language.csv', index=False)
movie_language_df.to_csv('movie_language.csv', index=False)

## 4. 电影标签维度表与关系表的构建

In [18]:
def clean_tag(t):
    if pd.isna(t):
        return None
    s = str(t).strip()
    # 去掉类似 "['经典'" 这样的前缀/后缀
    s = s.strip("[]'\" ")
    # 过滤纯数字（年份等）
    if re.fullmatch(r'\d{2,4}', s):
        return None
    return s or None

tag_exploded = (
    raw_df[['movie_id', 'tags']]
      .explode('tags')
      .dropna(subset=['tags'])
)

tag_exploded['tag_clean'] = tag_exploded['tags'].apply(clean_tag)
tag_exploded = tag_exploded.dropna(subset=['tag_clean'])

# 同一电影内部去重
tag_exploded = tag_exploded.drop_duplicates(
    subset=['movie_id', 'tag_clean']
)

# 构造 tag 维度表
tag_dim = (
    tag_exploded[['tag_clean']]
      .drop_duplicates()
      .reset_index(drop=True)
)
tag_dim['tag_id'] = tag_dim.index + 1
tag_dim = tag_dim.rename(columns={'tag_clean': 'tag_name'})

# 构造 movie_tag 关系表
movie_tag_df = tag_exploded.merge(
    tag_dim,
    left_on='tag_clean', right_on='tag_name'
)[['movie_id', 'tag_id']]

In [52]:
tag_dim.to_csv('tag.csv', index=False)
movie_tag_df.to_csv('movie_tag.csv', index=False)

## 5. 电影外部播放平台维度表和关系表的构建

In [21]:
platform_rows = []

for _, row in raw_df.iterrows():
    mid = row['movie_id']
    for v in (row.get('videos') or []):
        src = v.get('source') or {}
        platform_rows.append({
            'movie_id': mid,
            'platform_literal': src.get('literal'),
            'platform_name': src.get('name'),
            'need_pay': v.get('need_pay'),
            'sample_link': v.get('sample_link')
        })

platform_df = pd.DataFrame(platform_rows)

# 构造 platform 维度表
platform_dim = (
    platform_df[['platform_literal', 'platform_name']]
      .drop_duplicates()
      .reset_index(drop=True)
)
platform_dim['platform_id'] = platform_dim.index + 1

# 构造 movie_platform 关系表
movie_platform_df = platform_df.merge(
    platform_dim,
    on=['platform_literal', 'platform_name']
)[['movie_id', 'platform_id', 'need_pay', 'sample_link']]
movie_platform_df = movie_platform_df.drop_duplicates()

In [53]:
platform_dim.to_csv('platform.csv', index=False)
movie_platform_df.to_csv('movie_platform.csv', index=False)